In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
import os
from dotenv import load_dotenv
import json
import requests

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
cursor = db["cleancompanies"].find({})
dfCleanComp = pd.DataFrame(cursor)

dfCleanComp

,_id,name,category_code,total_money_raised,founded_year,description,address1,address2,zip_code,city,state_code,country_code,location
0,5e3ff8d234cb8a83c36a1e7b,Omnidrive,network_hosting,$800k,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None
1,5e3ff8d234cb8a83c36a1e7c,Wetpaint,web,$39.8M,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,5e3ff8d234cb8a83c36a1e7d,Wetpaint,web,$39.8M,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
3,5e3ff8d234cb8a83c36a1e7e,Zoho,software,$0,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
4,5e3ff8d234cb8a83c36a1e7f,Digg,news,$45M,2004.0,None,135 Mississippi St,,94107,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.394523,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21756,5e3ff8d334cb8a83c36a7377,Oriact,software,$0,2009.0,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,"{'type': 'Point', 'coordinates': [8.4371634, 4..."
21757,5e3ff8d334cb8a83c36a7378,Willdan Group,other,$0,NaN,,2401 East Katella Avenue,Suite 300,92806-5909,Anaheim,CA,USA,"{'type': 'Point', 'coordinates': [-117.882314,..."
21758,5e3ff8d334cb8a83c36a7379,goBookmaker,web,$0,NaN,None,None,None,None,None,None,None,None
21759,5e3ff8d334cb8a83c36a737a,AfterLogic,software,$0,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,"{'type': 'Point', 'coordinates': [-74.3235539,..."


In [7]:
import os
from dotenv import load_dotenv

import requests

load_dotenv()

token = os.getenv("API_KEY_GOOGLE")

url=f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={token}"
res = requests.get(url)


In [95]:
#geocode de Viena 
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    print(data)
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

Viena = geocode("Vancouver")
Viena

{'standard': {'addresst': {}, 'city': 'Vancouver', 'prov': 'CA', 'countryname': 'Canada', 'postal': {}, 'confidence': '0.90'}, 'longt': '-123.10307', 'alt': {}, 'elevation': {}, 'latt': '49.24843'}


{'type': 'Point', 'coordinates': [-123.10307, 49.24843]}

In [49]:
def coordCity(city):
    geo = requests.get(f"https://geocode.xyz/{city}?json=1").json()
    coords = (geo["latt"],geo["longt"])
    return coords

In [50]:

Vancouver= 49.24843,-123.10307 

('48.20762', '16.36364')

In [3]:
def parameters(tipo,radius,citycoords):
    loc = ','.join(citycoords)
    endpoint = f"location={loc}&radius={radius}&type={tipo}"
    return endpoint

params = parameters('school',5000,('49.24843','-123.10307'))
params

'location=49.24843,-123.10307&radius=5000&type=school'

In [7]:
def requestNearby(endpoint):
    token = os.getenv("API_KEY_GOOGLE")
    if not token:
        raise ValueError("You must set a API_KEY_GOOGLE token")
        
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?{endpoint}&key={token}"
    
    res = requests.get(url)
    
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
        
    
    return res.json()

In [8]:
schools = requestNearby(params)

In [69]:
schools

{'html_attributions': [],
 'next_page_token': 'CqQCHQEAANcM_08nSktV8esbYZ6aYdyrqJd5UpzoRsZQn7rFkV-FbAp5815IEP1uA3dJq54C03PLXtMjvujMnti00eR3fUEeXInIqMMLTfO1RKBFYwdcwopU0IbhiymKySDbPGVio1c2r9TPb2ksUKegDVpeKpFCNCbAYzfwdPbCNc8SXJbftwbds4jU6nzLPdNIv2wANBONlBK61AGwuvHzDXngw3FjzDhm6XogftZJBbUVyQK2aio_zvobpq6NrdhnY0PGCKP21A-3DcW1Q4FUzwIIHb-qkK-MNyY1A3rgxKvfJRLF48BPPWv6hyuCtOEl4WpZ1SZWDX23J8ha-wFk7bx6wDBI4jdpfGiHR5Q5iMmQTfUEFIhg0efX2kJq7CVcsh9xsRIQJYaPFleDb-XTo31QQzAVlRoU7qxuIYrvOzZ2CdtqjB2YahvnczU',
 'results': [{'geometry': {'location': {'lat': 48.1823768, 'lng': 16.3494907},
    'viewport': {'northeast': {'lat': 48.1837055302915,
      'lng': 16.3507786802915},
     'southwest': {'lat': 48.18100756970851, 'lng': 16.3480807197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png',
   'id': 'cc6379d8560f561b0ba580d9a17a3f52bf6e9a92',
   'name': 'VHS polycollege Johannagasse',
   'photos': [{'height': 2160,
     'html_attributions': ['<a href="https://maps.google.c

In [70]:
df = pd.DataFrame(schools['results'])

In [71]:
df

,geometry,icon,id,name,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,opening_hours
0,"{'location': {'lat': 48.1823768, 'lng': 16.349...",https://maps.gstatic.com/mapfiles/place_api/ic...,cc6379d8560f561b0ba580d9a17a3f52bf6e9a92,VHS polycollege Johannagasse,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJ782TBTCobUcRwyEd41Gvwl4,"{'compound_code': '58JX+XQ Vienna, Austria', '...",3.3,ChIJ782TBTCobUcRwyEd41Gvwl4,GOOGLE,"[school, point_of_interest, establishment]",15,"Johannagasse 2, Wien",NaN
1,"{'location': {'lat': 48.1953647, 'lng': 16.385...",https://maps.gstatic.com/mapfiles/place_api/ic...,f5ebf70d6be8586198f9ab509446c37779d3e839,Privates Gymnasium Sacré Coeur Wien,"[{'height': 1535, 'html_attributions': ['<a hr...",ChIJZRMe1GQHbUcRR8aa6GXVkes,"{'compound_code': '59WP+45 Vienna, Austria', '...",3.8,ChIJZRMe1GQHbUcRR8aa6GXVkes,GOOGLE,"[secondary_school, school, point_of_interest, ...",37,"Rennweg 31, Wien",{'open_now': False}
2,"{'location': {'lat': 48.1951981, 'lng': 16.392...",https://maps.gstatic.com/mapfiles/place_api/ic...,a53e3c152651c88870669dd5b6c8a934d773975e,BG & BRG Wien 3,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJQV4S12UHbUcRKDOHst1Td0c,"{'compound_code': '59WR+3V Vienna, Austria', '...",4.4,ChIJQV4S12UHbUcRKDOHst1Td0c,GOOGLE,"[secondary_school, school, point_of_interest, ...",33,"Boerhaavegasse 15, Wien",NaN
3,"{'location': {'lat': 48.184727, 'lng': 16.3687...",https://maps.gstatic.com/mapfiles/place_api/ic...,8c31fc550e0fbc077d107e0d567fca8e884ddafe,Sir-Karl-Popper-Schule,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJlTDMhNOpbUcRMTErTyeQiO0,"{'compound_code': '59M9+VF Vienna, Austria', '...",3.9,ChIJlTDMhNOpbUcRMTErTyeQiO0,GOOGLE,"[secondary_school, school, point_of_interest, ...",41,"Wiedner Gürtel 68, Wien",NaN
4,"{'location': {'lat': 48.19074610000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,19e107b63c92eb44c38caef4c99e859c1ff350d6,HTL Rennweg,"[{'height': 2976, 'html_attributions': ['<a hr...",ChIJk8NqA14HbUcROMBjLb7cJpM,"{'compound_code': '59RW+7W Vienna, Austria', '...",4.0,ChIJk8NqA14HbUcROMBjLb7cJpM,GOOGLE,"[school, point_of_interest, establishment]",80,"Rennweg 89B, Wien",NaN
5,"{'location': {'lat': 48.1857445, 'lng': 16.356...",https://maps.gstatic.com/mapfiles/place_api/ic...,c136fed6ff3aee09526dda2e496331aaaef98c24,HTL Spengergasse,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJJXMjli6obUcROACyjJ8YEGs,"{'compound_code': '59P4+7M Vienna, Austria', '...",4.1,ChIJJXMjli6obUcROACyjJ8YEGs,GOOGLE,"[school, point_of_interest, establishment]",73,"Spengergasse 20, Wien",{'open_now': True}
6,"{'location': {'lat': 48.2403969, 'lng': 16.344...",https://maps.gstatic.com/mapfiles/place_api/ic...,dd1a22572c79c5eb1e68b3c00a30fa5fe5ea0c1e,BRG19,"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJs3zhZSkGbUcRG1iYkCbVwLA,"{'compound_code': '68RV+5W Vienna, Austria', '...",3.9,ChIJs3zhZSkGbUcRG1iYkCbVwLA,GOOGLE,"[school, point_of_interest, establishment]",17,"Krottenbachstraße 11-13, Wien",NaN
7,"{'location': {'lat': 48.2205158, 'lng': 16.372...",https://maps.gstatic.com/mapfiles/place_api/ic...,07968d9ef360edfab4db5aee3bee0ecc98eb10e5,VBS Augarten,"[{'height': 1417, 'html_attributions': ['<a hr...",ChIJeS8ZWKUHbUcRt0d3nwMo07o,"{'compound_code': '69CC+6V Vienna, Austria', '...",4.1,ChIJeS8ZWKUHbUcRt0d3nwMo07o,GOOGLE,"[school, university, point_of_interest, establ...",10,"Untere Augartenstraße 9, Wien",{'open_now': True}
8,"{'location': {'lat': 48.19870599999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,a13221265201372af966d58a66a2a2a68e063735,Fahrschule U3ver,"[{'height': 1152, 'html_attributions': ['<a hr...",ChIJf2bbEv8HbUcR3y3-UPkGnww,"{'compound_code': '58X9+FP Vienna, Austria', '...",4.1,ChIJf2bbEv8HbUcR3y3-UPkGnww,GOOGLE,"[school, point_of_interest, establishment]",21,"Hütteldorfer Straße 81B, Wien",{'open_now': True}
9,"{'location': {'lat': 48.2207672, 'lng': 16.359...",https://maps.gstatic.c

In [16]:
import json, requests
load_dotenv()
url = 'https://api.foursquare.com/v2/venues/explore'
clientid = os.getenv("CLIENT_ID")
clientsec = os.getenv('CLIENT_SECRET')
params = dict(
  client_id=clientid,
  client_secret=clientsec,
  v='20180323',
  near='Zurich',
  categoryId='4bf58dd8d48988d1d3941735'
)



In [20]:
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
data['response'].keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [68]:
data['response']['groups'][0].keys()

dict_keys(['type', 'name', 'items'])

In [78]:
vegan = pd.DataFrame(data['response']['groups'][0]['items'])

In [83]:
veganRes = vegan[["venue"]].apply(lambda r: r.venue ,result_type="expand", axis=1)
veganRes = veganRes[['name','location']]

In [84]:
veganLoc = veganRes[["location"]].apply(lambda r: r.location ,result_type="expand", axis=1)

In [85]:
veganLoc

,address,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood
0,Sihlstrasse 28,47.373173,8.536831,"[{'label': 'display', 'lat': 47.37317277157419...",8001,CH,Zürich,Zürich,Schweiz,"[Sihlstrasse 28, 8001 Zürich, Schweiz]",NaN,NaN
1,Seefeldstr. 2,47.365086,8.547895,"[{'label': 'display', 'lat': 47.36508558405570...",8008,CH,Zürich,Zürich,Schweiz,"[Seefeldstr. 2, 8008 Zürich, Schweiz]",NaN,NaN
2,Bahnhofstrasse 88,47.375686,8.539650,"[{'label': 'display', 'lat': 47.37568557778311...",8001,CH,Zürich,Zürich,Schweiz,"[Bahnhofstrasse 88, 8001 Zürich, Schweiz]",NaN,NaN
3,Europaallee,47.377723,8.535789,"[{'label': 'display', 'lat': 47.3777227152696,...",NaN,CH,Zürich,Zürich,Schweiz,"[Europaallee, Zürich, Schweiz]",NaN,NaN
4,Lintheschergasse 15,47.376238,8.538709,"[{'label': 'display', 'lat': 47.37623764527292...",8001,CH,Zürich,Zürich,Schweiz,"[Lintheschergasse 15, 8001 Zürich, Schweiz]",NaN,NaN
5,Feldstrasse 98,47.378753,8.523103,"[{'label': 'display', 'lat': 47.37875340408591...",8004,CH,Zürich,Zürich,Schweiz,"[Feldstrasse 98, 8004 Zürich, Schweiz]",NaN,NaN
6,Weinbergstrasse 15,47.378267,8.544294,"[{'label': 'display', 'lat': 47.37826692507983...",8001,CH,Zürich,Zürich,Schweiz,"[Weinbergstrasse 15, 8001 Zürich, Schweiz]",NaN,NaN
7,Tramstrasse 2,47.409065,8.546486,"[{'label': 'display', 'lat': 47.4090652961886,...",8050,CH,Zürich,Zürich,Schweiz,"[Tramstrasse 2, 8050 Zürich, Schweiz]",NaN,NaN
8,NaN,47.377899,8.526774,"[{'label': 'display', 'lat': 47.37789909948714...",NaN,CH,Zürich,Zürich,Schweiz,"[Zürich, Schweiz]",NaN,NaN
9,St. Annagasse 18,47.373536,8.536325,"[{'label': 'display', 'lat': 47.37353648234148...",8001,CH,NaN,NaN,Schweiz,"[St. Annagasse 18, 8001, Schweiz]",NaN,NaN


In [86]:
Veg = pd.concat([veganRes,veganLoc],axis=1)

In [89]:
Veg = Veg[['name','address','lat','lng']]

In [90]:
Veg

,name,address,lat,lng
0,Hiltl,Sihlstrasse 28,47.373173,8.536831
1,Tibits,Seefeldstr. 2,47.365086,8.547895
2,Dachterrasse Hiltl,Bahnhofstrasse 88,47.375686,8.539650
3,Hiltl Sihlpost,Europaallee,47.377723,8.535789
4,Roots and Friends,Lintheschergasse 15,47.376238,8.538709
5,Marktküche,Feldstrasse 98,47.378753,8.523103
6,Mohini,Weinbergstrasse 15,47.378267,8.544294
7,tibits,Tramstrasse 2,47.409065,8.546486
8,Hiltl Langstrasse,NaN,47.377899,8.526774
9,Hiltl Vegi Metzg,St. Annagasse 18,47.373536,8.536325


In [92]:
from cleaningFunc import asGeoJSON

In [93]:
Veg["location"] = Veg[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
Veg.

,name,address,lat,lng,location
0,Hiltl,Sihlstrasse 28,47.373173,8.536831,"{'type': 'Point', 'coordinates': [8.5368314644..."
1,Tibits,Seefeldstr. 2,47.365086,8.547895,"{'type': 'Point', 'coordinates': [8.5478950356..."
2,Dachterrasse Hiltl,Bahnhofstrasse 88,47.375686,8.539650,"{'type': 'Point', 'coordinates': [8.5396500613..."
3,Hiltl Sihlpost,Europaallee,47.377723,8.535789,"{'type': 'Point', 'coordinates': [8.5357889772..."
4,Roots and Friends,Lintheschergasse 15,47.376238,8.538709,"{'type': 'Point', 'coordinates': [8.5387086132..."
5,Marktküche,Feldstrasse 98,47.378753,8.523103,"{'type': 'Point', 'coordinates': [8.5231029051..."
6,Mohini,Weinbergstrasse 15,47.378267,8.544294,"{'type': 'Point', 'coordinates': [8.5442942077..."
7,tibits,Tramstrasse 2,47.409065,8.546486,"{'type': 'Point', 'coordinates': [8.5464864871..."
8,Hiltl Langstrasse,NaN,47.377899,8.526774,"{'type': 'Point', 'coordinates': [8.5267736185..."
9,Hiltl Vegi Metzg,St. Annagasse 18,47.373536,8.536325,"{'type': 'Point', 'coordinates': [8.5363251449..."
